Jupyter Notebook에서 pyspark을 사용하려면 별도의 환경변수 세팅이 필요합니다.

이 과정이 귀찮아서 저는 [findspark](https://github.com/minrk/findspark) 라이브러리를 사용합니다. findspark은 spark_home path를 자동으로 찾아줍니다. 

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
import random

### Spark Context

SparkContext = 스파크에 접속하자

In [2]:
sc = SparkContext()

Python의 list 느낌...?

In [16]:
nums = sc.parallelize([6,7,0,5])

take(num): num번째 까지의 element를 가져온다

In [68]:
nums.take(1)

[6]

In [69]:
nums.take(4)

[6, 7, 0, 5]

map을 이용해서 lambda expression을 적용할 수 있다.

In [27]:
squared = nums.map(lambda x: x*x).collect()
squared

[36, 49, 0, 25]

만약 Python list라면 lambda 없이 list comprehension을 이용하면 된다.

In [49]:
ls = [6, 7, 0, 5]
[l*l for l in ls]

[36, 49, 0, 25]

DataFrame을 만들어보자 

In [50]:
from pyspark.sql import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

> 🤔 값을 입력하려면 반드시 Tuble을 사용해야 하는가 ?

In [72]:
list_p = [
    ('John',19),
    ('Jam',28),
    ('Smith',29),
    ('Adam',35),
    ('Henry',50)
]
rdd = sc.parallelize(list_p)
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
DF_ppl = sqlContext.createDataFrame(ppl)

In [73]:
DF_ppl.take(1)

[Row(age=19, name='John')]

In [74]:
DF_ppl.take(5)

[Row(age=19, name='John'),
 Row(age=28, name='Jam'),
 Row(age=29, name='Smith'),
 Row(age=35, name='Adam'),
 Row(age=50, name='Henry')]

### Basic operation with PySpark

csv파일을 불러와서 기본적인 분석 과정을 밟아보자

In [75]:
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
sc.addFile(url)
sqlContext = SQLContext(sc)

In [85]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [86]:
df = sqlContext.read.csv(
    SparkFiles.get("adult_data.csv"), 
    header=True, 
    inferSchema=True
)

DataFrame의 스키마 설명을 보여준다.

In [101]:
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



요약 통계를 보여주긴 하는데,, 보기가 흉하다. 노트북이라서 그럴까 ? 

아래의 통계량을 보여준다.
- count
- mean
- standarddeviation
- min
- max

In [102]:
df.describe().show()

+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+-----------------+------------------+--------------+------+
|summary|                 x|               age|  workclass|            fnlwgt|   education|   educational-num|marital-status|      occupation|relationship|              race|gender|      capital-gain|     capital-loss|    hours-per-week|native-country|income|
+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+-----------------+------------------+--------------+------+
|  count|             48842|             48842|      48842|             48842|       48842|             48842|         48842|           48842|       48842|             48842| 48842|             48842|            48842|  

SQL에서 ``` SELECT * FROM [TABLE] LIMIT 5``` 같은 느낌

> 🤔 truncate의 True/False 차이는 ? 

In [82]:
df.show(5, truncate = True)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|  x|age|workclass|fnlwgt|   education|educational-num|    marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|  1| 25|  Private|226802|        11th|              7|     Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|
|  2| 38|  Private| 89814|     HS-grad|              9|Married-civ-spouse|  Farming-fishing|     Husband|White|  Male|           0|           0|            50| United-States| <=50K|
|  3| 28|Local-gov|336951|  Assoc-acdm|             12|Married-civ-spouse|  Protective-ser

변수의 포맷을 변경해야 할 경우 아래와 같이 수정할 수 있다.

In [103]:
from pyspark.sql.types import *

# income의 포맷을 string에서 float로 변경하자
df_chnage_type = df.withColumn('income', df['income'].cast(DoubleType()))

df_chnage_type.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: double (nullable = true)



PySpark에서 사용하는 타입은 아래와 같다 [참고](https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-types.html)

- BinaryType – Binary data.
- BooleanType – Boolean values.
- ByteType – A byte value.
- DateType – A datetime value.
- DoubleType – A floating-point double value.
- IntegerType – An integer value.
- LongType – A long integer value.
- NullType – A null value.
- ShortType – A short integer value.
- StringType – A text string.
- TimestampType – A timestamp value (typically in seconds from 1/1/1970).
- UnknownType – A value of unidentified type.

```SELECT COLUMN1, COLUMN2 FROM [TABLE] LIMIT 5```

In [99]:
df.select('age','fnlwgt').show(5)

+---+------+
|age|fnlwgt|
+---+------+
| 25|226802|
| 38| 89814|
| 28|336951|
| 44|160323|
| 18|103497|
+---+------+
only showing top 5 rows



``` SELECT education, count(*) as count FROM [TABLE] GROUP BY education ORDER BY count DESC ```

In [100]:
df.groupBy("education").count().sort("count",ascending=True).show()

+------------+-----+
|   education|count|
+------------+-----+
|   Preschool|   83|
|     1st-4th|  247|
|     5th-6th|  509|
|   Doctorate|  594|
|        12th|  657|
|         9th|  756|
| Prof-school|  834|
|     7th-8th|  955|
|        10th| 1389|
|  Assoc-acdm| 1601|
|        11th| 1812|
|   Assoc-voc| 2061|
|     Masters| 2657|
|   Bachelors| 8025|
|Some-college|10878|
|     HS-grad|15784|
+------------+-----+

